In [11]:
import sys
sys.path.append('../../')
import os

import pandas as pd
from utils import fill_empty, VColumns
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load input file

In [12]:
df1 = pd.read_csv("v1-start.csv")
df1.head()

,type,name,size,qty,eur_sales_price
0,SPARKLING,Prosecco Superiore Extra Dry Sui Nui,BOTTLE,19,27.00 €
1,SPARKLING,Südtiroler Sekt 600 Blanc de Blances Kellerei...,BOTTLE,7,56.00 €
2,SPARKLING,Südtiroler Sekt Pinot Noir 2016 Pfitscher Montan,BOTTLE,5,65.00 €
3,SPARKLING,Südtiroler Sekt Versé Rosé Pitzner Kardaun,BOTTLE,18,38.00 €
4,SPARKLING,Trentodoc Brut Nature Opera,BOTTLE,14,55.00 €


# Start elaborating columns

Create a new dataframe with empty values, so that we can start filling in values from the input dataframe, without overriding columns.

In [13]:
df1.head(2)

,type,name,size,qty,eur_sales_price
0,SPARKLING,Prosecco Superiore Extra Dry Sui Nui,BOTTLE,19,27.00 €
1,SPARKLING,Südtiroler Sekt 600 Blanc de Blances Kellerei...,BOTTLE,7,56.00 €


In [14]:
# create new empty dataframe
df = pd.DataFrame(columns=VColumns.v2())
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible


In [15]:
import re

# copy values from original columns to new columns
df['type'] = df1['type']
df['name'] = df1['name']
df['winery_name'] = "" # leave empty
df["vintage"] = df1["name"].apply(lambda x: int(re.search(r'\d{4}', x).group()) if re.search(r'\d{4}', x) else None) # extrapolate from name, where i match regex for 4 digits
df["quantity"] = df1["qty"]
df["visible"] = True

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,NaN,SPARKLING,Prosecco Superiore Extra Dry Sui Nui,,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,True
1,NaN,SPARKLING,Südtiroler Sekt 600 Blanc de Blances Kellerei...,,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,True
2,NaN,SPARKLING,Südtiroler Sekt Pinot Noir 2016 Pfitscher Montan,,NaN,NaN,2016.0,NaN,NaN,5,NaN,NaN,True
3,NaN,SPARKLING,Südtiroler Sekt Versé Rosé Pitzner Kardaun,,NaN,NaN,NaN,NaN,NaN,18,NaN,NaN,True
4,NaN,SPARKLING,Trentodoc Brut Nature Opera,,NaN,NaN,NaN,NaN,NaN,14,NaN,NaN,True


In [16]:
# size
df['size'] = df1["size"].fillna("BOTTLE")

# convert prices to cents
df['price'] = df1['eur_sales_price'].apply(lambda x: int(float(x.replace('€','').strip())*100)) # convert to cents
df['purchase_price'] = 0

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,NaN,SPARKLING,Prosecco Superiore Extra Dry Sui Nui,,NaN,BOTTLE,NaN,2700,0,19,NaN,NaN,True
1,NaN,SPARKLING,Südtiroler Sekt 600 Blanc de Blances Kellerei...,,NaN,BOTTLE,NaN,5600,0,7,NaN,NaN,True
2,NaN,SPARKLING,Südtiroler Sekt Pinot Noir 2016 Pfitscher Montan,,NaN,BOTTLE,2016.0,6500,0,5,NaN,NaN,True
3,NaN,SPARKLING,Südtiroler Sekt Versé Rosé Pitzner Kardaun,,NaN,BOTTLE,NaN,3800,0,18,NaN,NaN,True
4,NaN,SPARKLING,Trentodoc Brut Nature Opera,,NaN,BOTTLE,NaN,5500,0,14,NaN,NaN,True


In [17]:
df = fill_empty(df, VColumns.v2(), False)
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,,SPARKLING,Prosecco Superiore Extra Dry Sui Nui,,,BOTTLE,0,2700,0,19,,,True
1,,SPARKLING,Südtiroler Sekt 600 Blanc de Blances Kellerei...,,,BOTTLE,0,5600,0,7,,,True
2,,SPARKLING,Südtiroler Sekt Pinot Noir 2016 Pfitscher Montan,,,BOTTLE,2016,6500,0,5,,,True
3,,SPARKLING,Südtiroler Sekt Versé Rosé Pitzner Kardaun,,,BOTTLE,0,3800,0,18,,,True
4,,SPARKLING,Trentodoc Brut Nature Opera,,,BOTTLE,0,5500,0,14,,,True


# Merge files and write output

If there are multiple files, merge them into one and create single output file.

In [18]:
df.head()

df_out = pd.concat([df], ignore_index=True)
df_out.to_csv("v2-cleaned.csv", index=False)

bool()

False